In [1]:
import numpy as np
import pandas as pd

from utils import get_by_expnames, pretty

In [24]:
# when disk is saturated: 
# how many cores: avg_cpu_ms / avg_wall_ms

data = [
    # baseline: take 16 threads
    ('redness', 'baseline', 2.93, 10.75, 25.7),
    ('hash', 'baseline', 2.9, 10.35, 25.79),
    ('resnet10', 'baseline', 5.09, 7.08, 14.78),
    # smart: take 8 threads
    ('redness', 'smart', .78, 4.54, 1328.4),
    ('hash', 'smart', .74, 4.03, 1408.6),
    ('resnet10', 'smart', .77, 3.54, 1358.36)
]

df = pd.DataFrame(data, columns=['workload', 'disk', 'avg_wall_ms', 'avg_cpu_ms', 'bandwidth'])
df['throughput'] = 1000. / df['avg_wall_ms']
df

,workload,disk,avg_wall_ms,avg_cpu_ms,bandwidth,throughput
0,redness,baseline,2.93,10.75,25.70,341.296928
1,hash,baseline,2.90,10.35,25.79,344.827586
2,resnet10,baseline,5.09,7.08,14.78,196.463654
3,redness,smart,0.78,4.54,1328.40,1282.051282
4,hash,smart,0.74,4.03,1408.60,1351.351351
5,resnet10,smart,0.77,3.54,1358.36,1298.701299


In [25]:
N_CPU = 16
SATA = 500
NVME2000 = 2000
NVME6000 = 6000

In [26]:
for disk, bus in zip(['baseline', 'smart', 'smart'], [SATA, NVME2000, NVME6000]):
    df1 = df[df['disk']==disk].copy()
    df1['n_disk_by_cpu'] = N_CPU*1e3 / (df1['avg_cpu_ms'] * df1['throughput'])
    df1['n_disk_by_bus'] = bus / df1['bandwidth']
    df1['n_disk'] = df1[['n_disk_by_cpu', 'n_disk_by_bus']].min(axis=1)
    df1['achievable_tput'] = df1['throughput'] * df1['n_disk']
    print df1[['workload', 'disk', 'n_disk_by_cpu', 'n_disk_by_bus', 'n_disk', 'achievable_tput']]

   workload      disk  n_disk_by_cpu  n_disk_by_bus     n_disk  \
0   redness  baseline       4.360930      19.455253   4.360930   
1      hash  baseline       4.483092      19.387359   4.483092   
2  resnet10  baseline      11.502825      33.829499  11.502825   

   achievable_tput  
0      1488.372093  
1      1545.893720  
2      2259.887006  
   workload   disk  n_disk_by_cpu  n_disk_by_bus    n_disk  achievable_tput
3   redness  smart       2.748899       1.505571  1.505571      1930.218732
4      hash  smart       2.937965       1.419849  1.419849      1918.715535
5  resnet10  smart       3.480226       1.472364  1.472364      1912.160692
   workload   disk  n_disk_by_cpu  n_disk_by_bus    n_disk  achievable_tput
3   redness  smart       2.748899       4.516712  2.748899      3524.229075
4      hash  smart       2.937965       4.259548  2.937965      3970.223325
5  resnet10  smart       3.480226       4.417091  3.480226      4519.774011
